In [56]:
import polars as pl

In [57]:
dfOutputs = pl.read_csv("../data/outputs.csv")

In [58]:
import re
from datetime import datetime

def extractDatetime(text):
    pattern = r'(\d{1,2}\.\s?\d{1,2}\.\s?\d{2,4})\s+(\d{1,2}\.\d{2})'

    matches = re.search(pattern, text)

    if matches:
        rawDate = matches.group(1)
        rawTime = matches.group(2)

        #date = datetime.strptime(rawDate, '%d. %m. %Y').strftime('%Y-%m-%d')
        #time = datetime.strptime(rawTime, '%H.%M').strftime('%H:%M')

        # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
        
        try:
            dt = datetime.strptime(rawDate.replace(" ", "") + " - " + rawTime, '%d.%m.%Y - %H.%M').strftime('%Y-%m-%d %H:%M')
        except ValueError:
            try:
                dt = datetime.strptime(rawDate.replace(" ", "") + " - " + rawTime, '%d.%m.%y - %H.%M').strftime('%Y-%m-%d %H:%M') # year with two digits
            except ValueError:
                dt = "00-00-0000 00:00"

        #print(f"Date: {date}")
        #print(f"Time: {time}")
        return dt
    else:
        #print("No date and time found.")
        return "00-00-0000 00:00"

In [59]:
dfOutputs = dfOutputs.with_columns([
    (pl.col("content").map_elements(lambda x: extractDatetime(x), return_dtype=pl.String).alias('time'))
])

In [60]:
dfOutputs.head()

year,month,file_name,content,time
i64,str,str,str,str
2022,"""April""","""TMP-1""","""Prometne informacije 30.…","""2022-04-30 18:30"""
2022,"""April""","""TMP-10""","""Prometne informacije 30.…","""2022-04-30 13:00"""
2022,"""April""","""TMP-100""","""Prometne informacije 27.…","""2022-04-27 06:30"""
2022,"""April""","""TMP-101""","""Prometne informacije 27.…","""2022-04-27 06:00"""
2022,"""April""","""TMP-102""","""Prometne informacije 26.…","""2022-04-26 20:00"""


In [61]:
dfOutputs.write_csv("../data/outputs.csv")